# Создал этот ноутбук для проверки простеньких сеток на синтетических датасетах / кусках датасетов

- Векторы V все почти что коллинеарны, когда их много - стоит уже посмотреть на $p_i$ и $r_{cut_{i}}$

- 

In [185]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

- СЕТКА С БОЛЬШИМ БАТЧ САЙЗ
- ГПР С ВЫРЕЗАННЫМ
- СЕТКА С ВЫРЕЗАННЫМ

- ОЧЕНЬ БЛИЗКИЕ ТОЧКИ - БЕРЕМ ЧЕТНЫЕ СТРОЧКИ ДЛЯ ТРЕЙНА И НЕЧЕТНЫЕ ДЛЯ ВАЛИДАЦИИ
- ТАК КАК ТУТ У НАС ДАННЫЕ ГДЕ ЧАСТИЦЫ НЕ ВРАЩАЮТСЯ, ТО МЫ МОЖЕМ ПРЕДСКАЗЫВАТЬ СРАЗУ ТРЕХМЕРНЫЙ ВЕКТОР - СТОИТ ПОСТРОИТЬ ЗАВИСИМОСТЬ 

In [186]:
def function(X):
    return 5 * X

In [187]:
path = '4_dataset_K_3.pt'

In [188]:
class CFG:
    '''

    All hyperparameters are here

    '''

    N = int(path.split('_')[0])     # число атомов
    K = int(path.split('_')[-1].split('.')[0])     # можно называть это разрешением...чем число больше, тем больше размеры матрицы для атомов, фактически это число элементов в наборах p и r_cut

    L = 2 * N ** (1 / 3) # размер одной клетки при моделировании

    r_cut = np.random.uniform(low=5, high=10, size=K).copy()
    p = np.random.uniform(low=1, high=3, size=K).copy()
    N_neig= N - 1 if N != 2 else 1

    # train_bs = 8
    # val_bs = 16
    batch_size = 32

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    f_threshold = 5    # Если сила по какой-то координате превышает это значение, то строчка исключается, совсем маленьких по модулю сил быть не должно, если что при генерации просто r_cut поменьше надо делать
    coord_threshold = L     # Если вдруг очень большие расстояния, то надо выкидывать
    f_min_threshold = 0.05
    #
    output_size = K     # Размерность аутпута модели

In [189]:
def create_dataloaders(train_dataset, val_dataset, train_bs=CFG.batch_size, val_bs=CFG.batch_size, fold=None):
    '''

    Returns train_loader, val_loader

    fold: will be used in cross validation, when I will implement it

    '''
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=train_bs, shuffle=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=val_bs, shuffle=False)

    return train_loader, val_loader

In [190]:
def plot_k_dim_f_norm_from_x_norm(data_sample, figsize=(15, 10), fontsize=18):
    '''

    Функция строит зависимость таргета от X

    data_sample - итерируемый объект из тьюплов вида: (x, f_k_dim, f_3d, pinv_A)

    '''
    plt.figure(figsize=figsize)
    x = torch.stack([elem[0][0].squeeze() for elem in data_sample])

    y = torch.stack([elem[1][0].squeeze() for elem in data_sample])

    plt.scatter(x, y, label='y(x)')
    plt.title('Зависимость $Y_{target}(X)$', fontsize=fontsize)

    None

In [191]:
# class MultiOutputMSELoss(nn.MSELoss):
#     '''

#     Custom loss that calculates average over batch loss for multidim MSE - sum of MSE for components

#     Example:
#     |
#     |    Loss = MultiOutputMSELoss()
#     |
#     |    a = torch.ones((8, 3))      # it is batch of 8 vectors of size 3
#     |    b = torch.zeros((8, 3))
#     |
#     |    Loss(a, b, batch_size=8) -> 3

#     '''

#     def forward(self, input, target, batch_size=CFG.batch_size):
#         '''
#         оно при reduction='mean' делит на произведение всех размерностей
#         '''
#         # при очень большом размере батча последние батчи будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера

#         return F.mse_loss(input, target, reduction='sum') / input.size(0)   # или эквивалентно делать reduction='mean' и умножать на input.size()[-1] - length of output

In [192]:
def make_predictions_and_plot(model, X_matrices=None, Y_target=None, figsize=(30, 20), fontsize=20, criterion=nn.MSELoss(), data=None, same_axis=False):
    '''

    Строит предсказанную и тагрет зависимости

    Можно подавать либо отлельно X_matrices, Y_target либо подать датасет из тьюплов: (X, f_k_dim, f_3d, A_pinv)

    Будет подаваться 3 итерируемых объекта: набор(батч или кусок датасета) из "матриц", предсказания, таргеты

    Качество

    '''

    if data:
        X_matrices, Y_target, _, _ = list(map(lambda x: torch.stack(x), list(zip(*data))))
        
    Y_pred = model(X_matrices)
    metric = criterion(Y_pred, Y_target)
    names = ['predicted', 'target']
    Ys = [Y_pred, Y_target]
    metric = criterion(Y_pred, Y_target)
    
    if CFG.K != 1:
        print(
            f'Metric: {metric}'
        )
        return

    plt.figure(figsize=figsize)
    fig = plt.gcf()
    fig.suptitle(f'Metric: {metric}', fontsize=22)

    for (i, name) in enumerate(names):
        if not same_axis:
            plt.subplot(1, 2, i + 1)

        plt.scatter(X_matrices.squeeze().detach().numpy(), Ys[i].squeeze().detach().numpy(), label='Y(X)')

        plt.xlabel('X', fontsize=18)
        plt.ylabel('Y', fontsize=18)
        plt.legend(loc='best', fontsize=fontsize)
        plt.title(f'{name}', fontsize=18)
        plt.grid(alpha=0.4)

    plt.show()

In [193]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality):
    '''

    Функция обучения по всем батчам 1 раз (1 эпоха)

    scaler: gradient scaler from torch.amp, попозже добавлю обучение с ним

    В данной версии: (X, f_k_dim)

    Лосс выводится для k-мерного предсказания, а mse считается по 3D вариантам, однако при K=1 3d и 1d MSE совпадают

    '''
    model.train()

    running_loss = 0.0
    running_MSE = 0
    processed_size = 0

    for inputs, labels, pinv_As, labels_3D in train_loader:
        optimizer.zero_grad()

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()

        if show_3D_quality:
            # print(pinv_As.size(), torch.unsqueeze(outputs.to(torch.float), 2).size())
            outputs_3D = torch.bmm(pinv_As, torch.unsqueeze(outputs.to(torch.float), 2)).to(CFG.device)    # используются для вычисления MSE метрики уже на 3D векторах силы
            outputs_3D = torch.squeeze(outputs_3D, -1)
            running_MSE += F.mse_loss(input=outputs_3D, target=labels_3D, reduction='sum').item()

        running_loss += loss.item() * inputs.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += inputs.size(0)

    # print(labels)
    train_loss = running_loss / processed_size
    if show_3D_quality:
        train_MSE = running_MSE / processed_size
        return train_loss, train_MSE
    
    return train_loss

In [194]:
def eval_epoch(model, val_loader, criterion, show_3D_quality):
    '''

    Одна эпоха по val выборке

    '''

    model.eval()
    
    running_loss = 0.0
    running_MSE = 0
    processed_size = 0

    for inputs, labels, pinv_As, labels_3D in val_loader:

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        if show_3D_quality:
            outputs_3D = torch.bmm(pinv_As, torch.unsqueeze(outputs.to(torch.float), 2)).to(CFG.device)    # используются для вычисления MSE метрики уже на 3D векторах силы
            outputs_3D = torch.squeeze(outputs_3D, -1)
            running_MSE += F.mse_loss(input=outputs_3D, target=labels_3D, reduction='sum').item()

        running_loss += loss.item() * inputs.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += inputs.size(0)

    # print(f' outputs:\n{outputs}, \n labels: \n {labels}')
    
    val_loss = running_loss / processed_size
    if show_3D_quality:
        val_MSE = running_MSE / processed_size
        return val_loss, val_MSE

    return val_loss

In [195]:
def train(train_loader, val_loader, model, optimizer, scheduler=None, epochs=10, scaler=None, criterion=nn.MSELoss(), show_3D_quality=False):
    '''

    Basic option: calculation loss on K-dimensional outputs, but MSE metric on 3D outputs, after the matrix is applied

    loss_on_k_projections: calculate loss'
    
    '''

    history = []
    if show_3D_quality:
        log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} val_loss: {v_loss:0.4f} train_mse: {t_mse:0.4f} val_mse: {v_mse:0.4f}"
    else:
        log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} val_loss: {v_loss:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            if show_3D_quality:
                train_loss, train_mse = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality)
                val_loss, val_mse = eval_epoch(model, val_loader, criterion, show_3D_quality)
            else:
                train_loss = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality)
                val_loss = eval_epoch(model, val_loader, criterion, show_3D_quality)
            history.append((train_loss, val_loss))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss, v_loss=val_loss))
            
    return history

In [196]:
class SingleNet(nn.Module):
    '''

    Класс одиночной нейронной сети

    '''
    def __init__(self, output_size, activation=nn.ReLU(), flattened_size=CFG.K * CFG.K):
        '''
        
        FC_type: тип полносвязных слоев: 'regular' / 'simple

        convolution: сверточная часть сети

        '''
        super().__init__()

        self.FC = nn.Sequential(
            nn.Linear(flattened_size, 64),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(64),

            nn.Linear(64, 128),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),
            nn.Linear(256, output_size),
        )

        # self.FC = nn.Linear(flattened_size, output_size)

    def forward(self, x):
        # x - is batch of matrices KxK

        # Здесь происходят какие-то там свертки, пуллинги и тп..

        x = self.FC(x)

        return x

In [199]:
def recieve_loaders(take_one_projection_for_data=None, path=None, cut_size=None):
    '''
    returns: (dataet, train_loader, val_loader), if u pass path -> returns loader from tensor dataset from

    может создать датасет у которого всего одна проекция взята за таргет из уже существующего датасета или по пути на файл:
    take_one_projection_for_data - указываем в этом параметре номер координатной проекции (нумерация от 1)
    '''
    if path:
        N = int(path.split('_')[0])
        K = int(path.split('_')[-1].split('.')[0])

        dataset = torch.load(str(N) + '_dataset_K_' + str(K) + '.pt')
        dataset = [(elem[0], elem[1], elem[2], elem[3]) for elem in dataset]

        if take_one_projection_for_data:
            dataset = [(elem[0], elem[1][take_one_projection_for_data - 1].unsqueeze(dim=0), elem[2], elem[3]) for elem in dataset]

        train_data, val_data = train_test_split(dataset, test_size=0.33, random_state=42)
        if cut_size:
            train_data = train_data[:cut_size]
            val_data = val_data[:cut_size]
        train_dataloader, val_dataloader = create_dataloaders(train_data, val_data)
        return train_data, val_data, train_dataloader, val_dataloader
    
    X = []
    Y = []

    for _ in range(500):
        # X = (torch.rand(1)).squeeze().unsqueeze(dim=0)
        x = (torch.rand(K))
        y = function(x)
        X.append(x)
        Y.append(y)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=42, train_size=0.8)

    X_train = torch.stack(X_train)
    X_val = torch.stack(X_val)
    Y_train = torch.stack(Y_train)
    Y_val = torch.stack(Y_val)

    train_data = TensorDataset(X_train, Y_train)
    val_data = TensorDataset(X_val, Y_val)

    train_dataloader = DataLoader(train_data, batch_size=128)
    val_dataloader = DataLoader(val_data, batch_size=128)

    return train_data, val_data, train_dataloader, val_dataloader

In [200]:
train_data, val_data, train_dataloader, val_dataloader = recieve_loaders(
    take_one_projection_for_data=1, #DONT WRITE 0 HERE: THERE'S A BUG
    path=path,
    cut_size=None
    )

In [201]:
val_data

[(tensor([1.2397, 0.0000, 0.0000, 0.0000, 0.1973, 0.0000, 0.0000, 0.0000, 1.9469]),
  tensor([0.0457]),
  tensor([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]),
  tensor([0.0457, 0.1349, 0.1153])),
 (tensor([1.2400, 0.0000, 0.0000, 0.0000, 0.2041, 0.0000, 0.0000, 0.0000, 1.9524]),
  tensor([-0.1810]),
  tensor([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]),
  tensor([-0.1810, -0.1304, -0.1783])),
 (tensor([2.2169, 0.0000, 0.0000, 0.0000, 0.5185, 0.0000, 0.0000, 0.0000, 0.4345]),
  tensor([0.1819]),
  tensor([[-1.,  0.,  0.],
          [ 0.,  1.,  0.],
          [ 0.,  0.,  1.]]),
  tensor([-0.1819,  0.1575, -0.0639])),
 (tensor([0.8254, 0.0000, 0.0000, 0.0000, 1.4703, 0.0000, 0.0000, 0.0000, 1.5995]),
  tensor([-0.1853]),
  tensor([[ 1.,  0.,  0.],
          [ 0.,  1.,  0.],
          [ 0.,  0., -1.]]),
  tensor([-0.1853, -0.1517,  0.2330])),
 (tensor([2.2167, 0.0000, 0.0000, 0.0000, 0.5168, 0.0000, 0.0000, 0.0000, 0.4297]),
  tensor([0.2097]),
  tenso

Сама частица, отступаем от нее вектор силы

In [202]:
# N=2, K=1:

# flattened_size = CFG.K ** 2
# output_size = CFG.K

# model = nn.Sequential(
#     nn.Linear(flattened_size, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(32),

#     nn.Linear(32, 128),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(128),
#     nn.Linear(128, output_size),
#     ).to(CFG.device)

# optimizer = optim.Adam(model.parameters(), lr=5e-3, betas=(0.9, 0.999), weight_decay=0.01)

# exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [203]:
model = SingleNet(
    output_size=1,

    # activation=nn.Sigmoid(),
    activation=nn.ReLU(),

).to(CFG.device)

# model = nn.Linear(1, 1)

# model = nn.Sequential(
#     nn.Linear(CFG.K ** 2, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(32),

#     nn.Linear(32, CFG.K)
# )

optimizer = optim.Adam(model.parameters(), lr=5e-2, betas=(0.9, 0.999), weight_decay=0)

# scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

model

SingleNet(
  (FC): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [204]:
CFG.batch_size

32

In [206]:
(next(
    iter(train_dataloader)
)[1])

tensor([[-0.0576],
        [-0.2224],
        [ 0.0531],
        [ 0.0215],
        [ 0.0459],
        [-0.0320],
        [-0.2358],
        [-0.1136],
        [-0.1035],
        [-0.1065],
        [-0.0599],
        [ 0.2334],
        [ 0.1337],
        [ 0.0302],
        [-0.2232],
        [ 0.4233],
        [ 0.1217],
        [-0.0776],
        [ 0.0274],
        [-0.1907],
        [-0.0795],
        [ 0.1057],
        [-0.0970],
        [ 0.0480],
        [ 0.1063],
        [-0.0989],
        [-0.0918],
        [-0.2894],
        [-0.0636],
        [-0.0723],
        [ 0.0532],
        [ 0.1919]])

In [210]:
history = train(
    train_loader=train_dataloader, val_loader=val_dataloader, model=model, optimizer=optimizer,
    
    scheduler=exp_scheduler,
    scaler=None,
    
    # criterion=MultiOutputMSELoss(),     # on K=1 it's the same as nn.MSELoss
    criterion=nn.MSELoss(),
    # criterion = GaussianNLLLossWithReadyVar(),

    # show_3D_quality=True,    # ПОКА ЧИСТО ТЕСТЮ НА ОДНОЙ ПРОЕКЦИИ И ТАМ РАЗМЕРНОСТИ НЕ СОВПАДАЮТ
    
    epochs=10
)

epoch:  50%|█████     | 5/10 [00:00<00:00, 21.45it/s]


Epoch 001 train_loss: 0.0246 val_loss: 0.0236

Epoch 002 train_loss: 0.0241 val_loss: 0.0238

Epoch 003 train_loss: 0.0241 val_loss: 0.0235

Epoch 004 train_loss: 0.0231 val_loss: 0.0230

Epoch 005 train_loss: 0.0229 val_loss: 0.0233


epoch: 100%|██████████| 10/10 [00:00<00:00, 22.48it/s]


Epoch 006 train_loss: 0.0244 val_loss: 0.0233

Epoch 007 train_loss: 0.0236 val_loss: 0.0226

Epoch 008 train_loss: 0.0239 val_loss: 0.0233

Epoch 009 train_loss: 0.0267 val_loss: 0.0230

Epoch 010 train_loss: 0.0245 val_loss: 0.0226


In [211]:
def plot_history(history, figsize=(40, 15)):
    '''

    history: [(train_loss, train_MSE, val_loss, val_MSE), ...]

    '''

    names = ['Losses', 'Metrics']
    
    figure = plt.figure(figsize=figsize)

    train_loss, train_metric, val_loss, val_metric = list(zip(*history))

    parameters = {'Losses': [train_loss, val_loss], 'Metrics': [train_metric, val_metric]}

    for i, name in enumerate(names):

        plt.subplot(1, 2, i + 1)
        plt.plot(range(1, len(history) + 1), parameters[name][0], label='train_loss')
        plt.plot(range(1, len(history) + 1), parameters[name][1], label='val_loss')
        
        plt.title(names[i], fontsize=18)
        plt.xlabel('epoch', fontsize=16)
        plt.ylabel('value', fontsize=16)     
        plt.xticks(np.arange(1, len(history) + 1, 1))
        # plt.yticks(np.arange(1, len(history) + 1, 1))
        plt.legend(loc='best', fontsize=16)
        plt.grid(alpha=0.4)
    
    plt.show()

In [213]:
make_predictions_and_plot(model=model, data=val_data, same_axis=True)

Metric: 0.022575629875063896


In [182]:
plot_history(history=history)

ValueError: not enough values to unpack (expected 4, got 2)

<Figure size 2880x1080 with 0 Axes>

In [183]:
train_data[0]

(tensor([0.8240, 0.0000, 0.0000, 0.0000, 1.4698, 0.0000, 0.0000, 0.0000, 1.5963]),
 tensor([-0.0044]),
 tensor([[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0., -1.]]),
 tensor([-0.0044,  0.0805,  0.2801]))

In [184]:
x_t = next(iter(val_dataloader))[0]
y_t = next(iter(val_dataloader))[1]
y_t

tensor([[-0.1100],
        [-0.2514],
        [ 0.2810],
        [-0.1189],
        [ 0.1054],
        [-0.0542],
        [ 0.1351],
        [ 0.3330],
        [ 0.0495],
        [ 0.0594],
        [ 0.3721],
        [-0.2649],
        [ 0.0191],
        [ 0.0394],
        [-0.1451],
        [ 0.0297],
        [ 0.1840],
        [ 0.1413],
        [-0.0403],
        [ 0.2137],
        [-0.1198],
        [ 0.0951],
        [-0.3227],
        [ 0.4183],
        [-0.2114],
        [ 0.1724],
        [-0.0067],
        [-0.0274],
        [ 0.1344],
        [ 0.0218],
        [ 0.0924],
        [ 0.1247]])

In [ ]:
model(x_t)

tensor([[-0.0192,  0.0058,  0.0222],
        [-0.0191,  0.0590,  0.0774],
        [-0.0076,  0.0596,  0.0814],
        ...,
        [-0.0152,  0.0593,  0.0791],
        [-0.0172, -0.0007,  0.0393],
        [-0.0014,  0.0171,  0.0758]], grad_fn=<AddmmBackward0>)